In [1]:
#네이버 book api로 책 정보 가져오기

In [6]:
#.env에서 API키 가져오기
import os
from dotenv import load_dotenv
load_dotenv('.env')
CLIENT_ID = os.getenv('X_NAVER_CLIENT_ID')
CLIENT_SECRET = os.getenv('X_NAVER_CLIENT_SECRET')
# HTTP 요청 헤더 설정
headers = {
"X-Naver-Client-Id": CLIENT_ID,
"X-Naver-Client-Secret": CLIENT_SECRET
}

In [7]:
query = "원피스, 오다에이치로, 대원씨아이"

In [8]:
# API 요청 기본 URL
url = "https://openapi.naver.com/v1/search/book.json"

In [9]:
display = 100
start = 1  # 시작 위치
total_books = []  # 전체 데이터를 저장할 리스트

In [10]:
# 최대 3번 반복하여 300권 가져오기
import pandas as pd
import requests


for _ in range(3):  
    # API 요청 파라미터
    params = {
        "query": query,
        "display": display,  # 한 번에 가져올 개수
        "start": start,  # 시작 위치
        "sort": "date"  # 출간일순 정렬
    }
    # API 요청 보내기
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        
        # 데이터가 없으면 종료
        if not data["items"]:
            break
        
        # 도서 정보 추가
        for book in data["items"]:
            total_books.append({
                "저자": book["author"],
                "제목": book["title"],
                "ISBN": book["isbn"],
                "출판사": book["publisher"],
                "출간일": book["pubdate"]
            })
    
        
        # 다음 페이지로 이동
        start += display  

        # 최대 300권까지만 가져오도록 설정
        if len(total_books) >= 300:
            break
    
    else:
        print(f"오류 발생: {response.status_code}, {response.text}")
        break

df = pd.DataFrame(total_books)
df["출간일"] = pd.to_datetime(df["출간일"], format="%Y%m%d", errors="coerce")  # 날짜 변환
df = df.sort_values(by="출간일", ascending=False)  # 최신순 정렬


In [11]:
df

,저자,제목,ISBN,출판사,출간일
0,오다 에이치로,원피스 ONE PIECE 110 (시대의 일렁임),9791142305320,대원씨아이,2025-01-23
4,오다 에이치로,"원피스 세트 (105~110권,전6권)",,대원씨아이,2025-01-23
6,오다 에이치로,"원피스 세트 (51~110권,전60권)",,대원씨아이,2025-01-23
5,오다 에이치로,"원피스 세트 (101~110권,전10권)",,대원씨아이,2025-01-23
1,오다 에이치로,"원피스 세트 (1~110권,전110권)",,대원씨아이,2025-01-23
...,...,...,...,...,...
295,오다 에이치로,원피스 34 (’물의 도시’ 워터 세븐),9788925285870,대원씨아이,2011-08-23
296,오다 에이치로,원피스 59 (포트거스 D 에이스 죽다),9788925286068,대원씨아이,2011-08-23
297,오다 에이치로,원피스 36 (9번째 정의),9788925285887,대원씨아이,2011-08-23
298,오다 에이치로,원피스 58 (이 시대의 이름을 ’흰 수염’이라 부른다),9788925286051,대원씨아이,2011-08-23


In [12]:
#출간일 정보 부정확
#ISBN을 통해 가져온다.
#ISBN 정보가 없는 경우를 제외시킴, nan으로 저장이 안되었고 ""로 저장되어있음..
import numpy as np

# 빈 문자열("")을 NaN(np.nan)으로 변환
df["ISBN"] = df["ISBN"].replace("", np.nan)

df.isna().sum()

저자       0
제목       0
ISBN    37
출판사      0
출간일      0
dtype: int64

In [13]:
df.dropna(subset=["ISBN"], inplace=True)

In [14]:
df

,저자,제목,ISBN,출판사,출간일
0,오다 에이치로,원피스 ONE PIECE 110 (시대의 일렁임),9791142305320,대원씨아이,2025-01-23
7,오다 에이치로,원피스 110 + 2025 캘린더 합본판 (프로모션 동봉 카드 1종 + 스페셜 일러...,9791142305337,대원씨아이,2025-01-13
8,오다 에이치로,원피스 일러스트 캘린더(2025) (특제 디자인 카드 15매(부록)),8809894544094,대원씨아이,2024-11-20
9,"코지 소헤이 지음, 오다 에이치로 원작",원피스 학원 8 (코비의 꿈),9791172884956,대원씨아이,2024-11-18
10,오다 에이치로,원피스 109 (네 편),9791172884925,대원씨아이,2024-10-31
...,...,...,...,...,...
295,오다 에이치로,원피스 34 (’물의 도시’ 워터 세븐),9788925285870,대원씨아이,2011-08-23
296,오다 에이치로,원피스 59 (포트거스 D 에이스 죽다),9788925286068,대원씨아이,2011-08-23
297,오다 에이치로,원피스 36 (9번째 정의),9788925285887,대원씨아이,2011-08-23
298,오다 에이치로,원피스 58 (이 시대의 이름을 ’흰 수염’이라 부른다),9788925286051,대원씨아이,2011-08-23


In [15]:
#제목에 합본판이 들어간 책 제거
df = df[~df["제목"].str.contains("합본판")]

In [16]:
df

,저자,제목,ISBN,출판사,출간일
0,오다 에이치로,원피스 ONE PIECE 110 (시대의 일렁임),9791142305320,대원씨아이,2025-01-23
8,오다 에이치로,원피스 일러스트 캘린더(2025) (특제 디자인 카드 15매(부록)),8809894544094,대원씨아이,2024-11-20
9,"코지 소헤이 지음, 오다 에이치로 원작",원피스 학원 8 (코비의 꿈),9791172884956,대원씨아이,2024-11-18
10,오다 에이치로,원피스 109 (네 편),9791172884925,대원씨아이,2024-10-31
27,오다 에이치로,원피스 108 (죽는 편이 나은 세계),9791172039448,대원씨아이,2024-05-31
...,...,...,...,...,...
295,오다 에이치로,원피스 34 (’물의 도시’ 워터 세븐),9788925285870,대원씨아이,2011-08-23
296,오다 에이치로,원피스 59 (포트거스 D 에이스 죽다),9788925286068,대원씨아이,2011-08-23
297,오다 에이치로,원피스 36 (9번째 정의),9788925285887,대원씨아이,2011-08-23
298,오다 에이치로,원피스 58 (이 시대의 이름을 ’흰 수염’이라 부른다),9788925286051,대원씨아이,2011-08-23


In [17]:
df.to_csv("books.csv", index=False)

In [18]:
#https://www.nl.go.kr/NL/contents/N31101020000.do에서 ISBN으로 책 정보 가져오기

In [19]:
# ISBN 리스트 추출
isbn_list = df["ISBN"].dropna().unique().tolist()


In [20]:
isbn_list

['9791142305320',
 '8809894544094',
 '9791172884956',
 '9791172884925',
 '9791172039448',
 '9791171723331',
 '9791171723348',
 '9791171723300',
 '9791171248971',
 '9791168945401',
 '9791171240616',
 '9791168945449',
 '9791168945388',
 '9791168945302',
 '9791168945456',
 '9791168945425',
 '9791168945371',
 '9791168945357',
 '9791168945395',
 '9791169796002',
 '9791170620228',
 '9791168945203',
 '9791168945432',
 '9791169793124',
 '9791168945272',
 '9791168945333',
 '9791168945364',
 '9791168945296',
 '9791168945326',
 '9791168945289',
 '9791168945319',
 '9791169791533',
 '9791169791526',
 '9791168945227',
 '9791168945173',
 '9791168945234',
 '9791169446754',
 '9791168945265',
 '9791168945258',
 '9791168945210',
 '9791168945241',
 '9791168945197',
 '9791169441315',
 '9791169441322',
 '9791168945081',
 '9791168945135',
 '9791168945098',
 '9791168945128',
 '9791168945104',
 '9791168945166',
 '9791168945142',
 '9791168945159',
 '9791169180931',
 '9791168945005',
 '9791168945074',
 '97911689

In [ ]:
#카카오이용

In [39]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# .env에서 API 키 불러오기
load_dotenv()
KAKAO_API_KEY = os.getenv("KAKAO_REST_API_KEY")

# 검색할 만화 제목 (예: 원피스)
query = "노다메칸타빌레"

# API 요청 URL
url = "https://dapi.kakao.com/v3/search/book"

# 요청 헤더 설정
headers = {
    "Authorization": f"KakaoAK {KAKAO_API_KEY}"
}

# API 요청 파라미터
params = {
    "query": query,
    "size": 50,  # 최대 50개 결과 가져오기
    "sort": "recency"  # 최신순 정렬
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    books = []

    for book in data["documents"]:
        books.append({
            "제목": book["title"],
            "출간년월": book["datetime"][:7],  # YYYY-MM 형식
            "ISBN": book["isbn"]
        })

    # DataFrame 변환 및 정렬
    df = pd.DataFrame(books)
    df = df.sort_values(by="출간년월")

else:
    print(f"오류 발생: {response.status_code}, {response.text}")

In [40]:
df.sort_values(by="출간년월")

,제목,출간년월,ISBN
36,노다메 칸타빌레 1,2002-06,8952844106 9788952844101
11,のだめカンタ-ビレ 6* 노다메 칸타빌레. 6,2003-07,4063404382 9784063404388
41,노다메 칸타빌레 6,2003-09,8925225352 9788925225357
13,のだめカンタ-ビレ 7* 노다메 칸타빌레. 7,2003-10,406340451X 9784063404517
49,노다메 칸타빌레 7,2003-12,8952870182 9788952870186
37,노다메 칸타빌레 8,2004-05,8952876334 9788952876331
43,노다메 칸타빌레 8,2004-05,8925225379 9788925225371
7,のだめカンタ-ビレ 9* 노다메 칸타빌레. 9,2004-06,4063404889 9784063404883
47,노다메 칸타빌레 9,2004-08,8952881680 9788952881687
12,のだめカンタ-ビレ 10* 노다메 칸타빌레. 10,2004-09,4063405052 9784063405057


In [33]:
# 검색할 책 제목 (예: 원피스)
query = "만화,완결"

In [18]:
import requests
import pandas as pd
import os
import numpy as np
from dotenv import load_dotenv

# .env에서 API 키 불러오기
load_dotenv()
GOOGLE_BOOKS_API_KEY = os.getenv("GOOGLE_API_KEY") 

# API 요청 URL
url = "https://www.googleapis.com/books/v1/volumes"

# 가져올 총 개수 & 한 번에 가져올 개수 설정
total_results = 500  # 가져올 전체 개수
batch_size = 40  # Google Books API는 한 번에 최대 40개까지만 가져올 수 있음
start_index = 0  # 시작 위치

books = []

while start_index < total_results:
    # API 요청 파라미터 설정 (페이징 처리)
    params = {
        "q": query,
        "maxResults": batch_size,  # 한 번에 가져올 개수
        "startIndex": start_index,  # 가져올 데이터의 시작 위치
        "orderBy": "newest",  # 최신순 정렬
        "key": GOOGLE_BOOKS_API_KEY
    }

    # API 요청 보내기
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        items = data.get("items", [])

        if not items:
            break  # 더 이상 가져올 데이터가 없으면 중단

        for item in items:
            volume_info = item.get("volumeInfo", {})

            # ISBN 가져오기 (없으면 NaN)
            isbn_list = volume_info.get("industryIdentifiers", [])
            isbn = next((identifier["identifier"] for identifier in isbn_list if identifier["type"] == "ISBN_13"), np.nan)

            books.append({
                "제목": volume_info.get("title", "정보 없음"),
                "출간년월": volume_info.get("publishedDate", "정보 없음")[:7],  # YYYY-MM 형식
                "ISBN": isbn  # NaN 적용됨
            })

        # startIndex 증가 → 다음 데이터 요청
        start_index += batch_size  

    else:
        print(f"오류 발생: {response.status_code}, {response.text}")
        break  # 오류 발생 시 중단

# DataFrame 변환 및 정렬
df = pd.DataFrame(books)
df = df.sort_values(by="출간년월")

#제목에 query 중 1번이 안들어간건 제거 (ex. 원피스)
df = df[df["제목"].str.contains(query.split(",")[0])]

#제목에 세트, 합본판, 특별판 등이 들어간 책 제거
df = df[~df["제목"].str.contains("세트|합본판|특별판")]

In [19]:
df

,제목,출간년월,ISBN
63,한국 만화 의 선구자들,1995,NaN
130,성완경의 세계만화탐사(양장본 HardCover),2002,NaN
241,북유럽 신화 1(만화로보는),2002-08,9788979193817
115,일본 만화 의 사회학,2004,NaN
314,만화 삼국지 11,2007-01,9788959191109
263,만화 토지 6(박경리 원작),2007-05,9788960531062
283,만화 삼국지. 14: 운명에 맞서다(황석영 이충호의),2007-07,9788959191505
227,채운국 이야기. 1(만화),2008-07,9788953287501
111,신삼국지. 8: 천하통일편 (완결)(졸라맨 만화),2008-12,9788984922440
68,만화! 문화 사회학적 읽기,2009,9788973008087
